In [2]:
import pandas as pd  
from sklearn.tree import DecisionTreeClassifier  

In [3]:
clothing_train = pd.read_csv("clothing_data_driven_training (1)", sep="\t") 
clothing_test = pd.read_csv("clothing_data_driven_test", sep="\t") 

y = clothing_train[['Response']]
X = clothing_train[['Days since Purchase', 'Number of Purchase Visits', 'Sales per Visit']]

Upon completing the code and data sets that were posted under this week, continue using the below codes to construct the simplified data‐driven cost matrix as follows:  
a.	Obtain the mean of the Sales per Visit variable from the training data set and set the negative of that value to be the “cost” of a true positive. Let the false positive cost equal $10.  

In [6]:
# a  
sales_mean = round(clothing_train['Sales per Visit'].mean(),2)  
tp_cost = sales_mean - (sales_mean*2)  
fp_cost = 10
sales_mean


113.58

b.	Construct the appropriate cost matrix, and simplify it, to obtain the simplified data‐driven cost matrix. 

In [16]:
# b  
data = {'0': [0, 0],  
        '1': [fp_cost, tp_cost ]  
        }  
  
cost_matrix = pd.DataFrame (data, columns = ['0','1'])  

cost_matrix_adj = cost_matrix / fp_cost  

  
cost_matrix_adj.iloc[1,0] = cost_matrix_adj.iloc[1,1] - (cost_matrix_adj.iloc[1,1]*2)  
cost_matrix_adj.iloc[1,1] = 0  
cost_matrix_adj

,0,1
0,0.000,1.0
1,11.358,0.0


Using the training set, build a C5.0 model (Model 3) to predict a customer's Response using Days since Purchase, # of Purchase Visits, and Sales per Visit, using the data‐driven cost matrix.

In [37]:
cost_matrix = {0:cost_matrix_adj.iloc[0,1], 1:cost_matrix_adj.iloc[1,0]}  
  
# train C5.0 model  
c50_03 = DecisionTreeClassifier(criterion="entropy", min_samples_split=75, max_leaf_nodes=5 , class_weight=cost_matrix).fit(X,y)  

Populate the Model Evaluation Table with the evaluation measures for Model 3, using the data‐driven cost matrix. 

In [38]:
# use model to predict on test data  
c50_03_predict = c50_03.predict(clothing_test[['Days since Purchase', 'Number of Purchase Visits', 'Sales per Visit']])  
  
# construct contingency table  
con_table_3= pd.crosstab(clothing_test['Response'],c50_03_predict)
con_table_3

col_0,0,1
Response,,
0,4897,4717
1,266,1674


In [47]:
def model_eval_cost(matrix,model_name,fn_cost,tn_cost,fp_cost,tp_cost):  
#from a dataframe and iloc is Purely integer-location based indexing for selection by position.
    tn = matrix.iloc[0,0]  
    tp = matrix.iloc[1,1]  
    fn = matrix.iloc[1,0]  
    fp = matrix.iloc[0,1]  
#some formulas
    tap = fn+tp  
    tan = tn+fp  
    tpn = tn+fn  
    tpp = fp+tp  
    precision = tp/tpp  
    recall = tp/tap  
    total = tn+tp+fn+fp  
    data = [  
        round((tp+tn)/total,4),  #accuracy
        round(1-((tp+tn)/total),4),  #error rate
        round(tp/tap,4),  #Snesitivity
        round(tn/tan,4),  #Specificity / Recall
        round(precision,4),  #Precision
        round(2 * (precision * recall) / (precision + recall),4),	#F1
        round(5 * (precision * recall) / ((4 * precision) + recall),4),	#F2
        round(1.25 * (precision * recall) / ((.25 * precision) + recall),4),	#F3  
        round((fp*fp_cost)+(tp*tp_cost),2),  
        round(-1*((fp*fp_cost)+(tp*tp_cost))/total,4)   
        ]  
    return(  
        pd.DataFrame(data, columns=[model_name],   
        index=['Accuracy','Error Rate','Sensitivity','Specificity','Precision','F1','F2','F3','overall_cost','profit_per_customer'])  
        )  

Model_3 =  model_eval_cost(con_table_3, model_name = 'Model_3', fn_cost=0, tn_cost=0, fp_cost=10 , tp_cost=-113.58 )  
Model_3  


,Model_3
Accuracy,0.5687
Error Rate,0.4313
Sensitivity,0.8629
Specificity,0.5094
Precision,0.2619
F1,0.4019
F2,0.5915
F3,0.3043
overall_cost,-142962.9200
profit_per_customer,12.3735


In [ ]:
import datetime
import socket 
def get_Host_name_IP():
    try: 
        host_name = socket.gethostname() 
        host_ip = socket.gethostbyname(host_name) 
        print("Hostname:",host_name) 
        print("IP Address:",host_ip) 
    except: 
        print("No visible IP Address") 
get_Host_name_IP()
now = datetime.datetime.now()
print ("Time Stamp:", now.strftime("%Y-%m-%d %H:%M:%S"))